In [103]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn import metrics
from sklearn.metrics import pairwise_distances

In [105]:
product = pd.read_csv('/home/parul/Music/Practice/test/72cb9080-b-Q1ClData/cproducts.csv')
tender = pd.read_csv('/home/parul/Music/Practice/test/72cb9080-b-Q1ClData/ctender.csv')

In [107]:
product.head()

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
0,BBID_20482,1975-10-20,male,MADHYA PRADESH,453441.0,2016-05-01,2655,BB-INDORE-MALHAR MEGA MALL,2,85246,NONPROMO,no_promo,1000446431,PATANJALI CHOCOS 125g,55.00,Payback
1,BBID_20485,1955-07-21,female,no_state,999999.0,2017-03-09,2655,BB-INDORE-MALHAR MEGA MALL,8,78829,NONPROMO,no_promo,1000010431,SOO FRESH ROLLS HOT DOG 4P,30.00,Payback
2,BBID_20498,1981-03-19,female,MAHARASHTRA,400064.0,2015-08-12,2655,BB-INDORE-MALHAR MEGA MALL,13,1064,0000971754,AD 399 FLAT LADIES JEANS FASHION EXC,1000077851005,"DLJ-0010-BSF-2-CJ-FL, 34, BLACK",418.95,Payback
3,BBID_20499,1986-05-10,male,MAHARASHTRA,400099.0,2016-12-18,2615,BB-AMRITSAR-TRILIUM MALL,14,51080,NONPROMO,no_promo,1000443231,PATANJALI TOMATO KETCHUP BT 500g,70.00,Payback
4,BBID_204110,1988-02-29,male,MADHYA PRADESH,474001.0,2015-12-31,2655,BB-INDORE-MALHAR MEGA MALL,9,21911,0000974126,NMP 5% OFF CONCEPT DISCOUNT 2655,300481741,MAGGI NDL MERI MASALA PP 70g,34.20,Payback


In [109]:
tender.head()

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,tender_type,transaction_number_by_till,payment_amount_by_tender,PaymentUsed
0,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-03,4986,BB-MADURAI-KOCHADAI VILLAGE,22,GVOWN,9208,1000.00,GiftVoucher
1,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-03,4986,BB-MADURAI-KOCHADAI VILLAGE,22,CASH,9208,10.46,CASH
2,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,EDCAXIS,90269,677.28,AXIS
3,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,MOBI,90268,1042.00,One MobiKwik Systems
4,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,MOBI,90269,1000.00,One MobiKwik Systems


In [111]:
product['promotion_description'].fillna('no_promo', inplace=True)
product['Gender'].fillna('no_gender', inplace=True)
product['State'].fillna('no_state', inplace=True)
product['PinCode'].fillna(-1, inplace=True)
product['DOB'].fillna("1", inplace=True)

In [112]:
first = product.replace(['female', 'male'], [-1, 1])
second = first.replace(['BBProfitClub','BBProfitClub,Payback','BBSavingsClub','BBSavingsClub,BBProfitClub','BBSavingsClub,BBProfitClub,Payback','BBSavingsClub,Payback','BBSavingsClub,T24Club','BBSavingsClub,T24Club,BBProfitClub','BBSavingsClub,T24Club,BBProfitClub,Payback','BBSavingsClub,T24Club,Payback','FGShoppingFest','FGShoppingFest,BBProfitClub',
 'FGShoppingFest,BBProfitClub,Payback','FGShoppingFest,Payback','FGShoppingFest,T24Club','FGShoppingFest,T24Club,BBProfitClub','FGShoppingFest,T24Club,BBProfitClub,Payback',
 'FGShoppingFest,T24Club,Payback','FuturePrivilegeClub,BBProfitClub','FuturePrivilegeClub,FGShoppingFest','FuturePrivilegeClub,Payback',
 'FuturePrivilegeClub,T24Club,BBProfitClub','FuturePrivilegeClub,T24Club,BBProfitClub,Payback','FuturePrivilegeClub,T24Club,Payback','Payback',
 'T24Club,BBProfitClub','T24Club,BBProfitClub,Payback','T24Club,Payback'], [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])
second.head()

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
0,BBID_20482,1975-10-20,1,MADHYA PRADESH,453441.0,2016-05-01,2655,BB-INDORE-MALHAR MEGA MALL,2,85246,NONPROMO,no_promo,1000446431,PATANJALI CHOCOS 125g,55.00,24
1,BBID_20485,1955-07-21,-1,no_state,999999.0,2017-03-09,2655,BB-INDORE-MALHAR MEGA MALL,8,78829,NONPROMO,no_promo,1000010431,SOO FRESH ROLLS HOT DOG 4P,30.00,24
2,BBID_20498,1981-03-19,-1,MAHARASHTRA,400064.0,2015-08-12,2655,BB-INDORE-MALHAR MEGA MALL,13,1064,0000971754,AD 399 FLAT LADIES JEANS FASHION EXC,1000077851005,"DLJ-0010-BSF-2-CJ-FL, 34, BLACK",418.95,24
3,BBID_20499,1986-05-10,1,MAHARASHTRA,400099.0,2016-12-18,2615,BB-AMRITSAR-TRILIUM MALL,14,51080,NONPROMO,no_promo,1000443231,PATANJALI TOMATO KETCHUP BT 500g,70.00,24
4,BBID_204110,1988-02-29,1,MADHYA PRADESH,474001.0,2015-12-31,2655,BB-INDORE-MALHAR MEGA MALL,9,21911,0000974126,NMP 5% OFF CONCEPT DISCOUNT 2655,300481741,MAGGI NDL MERI MASALA PP 70g,34.20,24


In [113]:
q = list(set(product['DOB']))
third = second.replace(q, np.arange(len(q)))
l = list(set(product['customerID']))
fourth = third.replace(l, np.arange(len(l)))
m = list(set(product['State']))
fifth = fourth.replace(m, np.arange(len(m)))
n = list(set(product['store_description']))
sixth = fifth.replace(n, np.arange(len(n)))
o = list(set(product['promo_code']))
seventh = sixth.replace(o, np.arange(len(o)))
s = list(set(product['promotion_description']))
eight = seventh.replace(s, np.arange(len(s)))
t = list(set(product['product_description']))
ninth = eight.replace(t, np.arange(len(t)))

In [117]:
u = list(set(product['transactionDate']))
tenth = ninth.replace(u, np.arange(len(u)))

In [123]:
v = tenth.replace('no_gender', 0)

In [124]:
kmeans_model = KMeans(n_clusters=3, random_state=1).fit(v)
labels = kmeans_model.labels_
metrics.silhouette_score(v, labels, metric='euclidean')

0.99526435956394477

In [131]:
df1 = product[['customerID', 'store_code']]

In [133]:
label_column = pd.DataFrame(labels)

In [138]:
result=df1.join(label_column)
result.head()

,customerID,store_code,0
0,BBID_20482,2655,0
1,BBID_20485,2655,0
2,BBID_20498,2655,2
3,BBID_20499,2615,0
4,BBID_204110,2655,0


In [139]:
result.columns = ['customerID', 'store_code', 'cluster']
result.head()

,customerID,store_code,cluster
0,BBID_20482,2655,0
1,BBID_20485,2655,0
2,BBID_20498,2655,2
3,BBID_20499,2615,0
4,BBID_204110,2655,0


In [140]:
df = result.to_csv('result.csv', index=None)